## OpenStreetMap Data Wrangling with SQL

##### BY Sheng BI

---

### Project Overview

1. Select an area and download the data from [OpenStreetMap](https://www.openstreetmap.org).

2. Audit the data, investigate data using SQL and Pandas.


### Map Area  

West hollywood, CA, United States

This is a place where a lot of celebrities live, and I am interested in how do people live their life there. The data source is [OpenStreetMap](https://www.openstreetmap.org).

#### Importing necessary libraries. 

In [1]:
import os
import os.path
import sys
import requests                     ## for sending requests to specific url.
import xml.etree.ElementTree as ET  ## for parsing xml files
import pprint                       ## for pretty-printing certain data structures
import re                           ## for dealing with regular expressions.
import codecs
import json
from collections import defaultdict 
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import csv
import sqlite3
from datetime import datetime

----------------

### Part I: OSM Dataset

The following is an overview of the map.

In [2]:
from IPython.display import HTML

In [3]:
HTML('<iframe width="425" height="350" frameborder="0" scrolling="no" marginheight="0" marginwidth="0" src="http://www.openstreetmap.org/export/embed.html?bbox=-118.3918%2C34.0754%2C-118.3320%2C34.0996&amp;layer=mapnik"></iframe><br/>')

The data could
- either be obtained through the following link: 
<!--- https://overpass-api.de/api/map?bbox=-118.4172,34.0632,-118.2428,34.1425 --->
https://overpass-api.de/api/map?bbox=-118.3918,34.0754,-118.3320,34.0996
- or be downloaded via the following codes

In [ ]:
def download_file(url, local_filename):
    
    '''
    This function serves to download the osm_map file.
    
    The following link provides useful information:
    https://stackoverflow.com/questions/16694907/how-to-download-large-file-in-python-with-requests-py/16696317#16696317

    '''
    total_size = 58913*1024 # The size of the file to be downloaded is 58.90MB.
    
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        print "Start downloading {}".format(local_filename) 
        
        current_size = 0                               ## initialize size of file to be downloaded.
        
        for chunk in r.iter_content(chunk_size=4096): ## if the chunk_size is too small, error may occur 
                                                      ##  for reasons which I could not solve. 
            if chunk:
                f.write(chunk)
                current_size = current_size + len(chunk)
                downloaded_size = int(100 * current_size / float(total_size))
                if downloaded_size%10 == 0: 
                    sys.stdout.write("downloaded: {}{}\n".format(downloaded_size,"%"))
                sys.stdout.flush()

    print '\nDownload of {} is finished.'.format(local_filename)

In [ ]:
url = 'https://overpass-api.de/api/map?bbox=-118.4172,34.0632,-118.2428,34.1425'
filename = 'westhollywood.osm'

download_file(url, filename)

The following code (from Udacity website) allows us to extact a sample of the file.  

In [4]:
OSM_FILE = 'westhollywood.osm'  # Replace this with your osm file
SAMPLE_FILE = "sample.osm"

k = 100 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

One extracted sample node is as follows:

```xml
<node changeset="52419516" id="5133454776" lat="34.0928719" lon="-118.3790788" timestamp="2017-09-27T17:01:30Z" uid="3232636" user="doktorpixel14" version="1">
    <tag k="addr:city" v="West Hollywood" />
    <tag k="addr:housenumber" v="8591" />
    <tag k="addr:postcode" v="90069" />
    <tag k="addr:state" v="CA" />
    <tag k="addr:street" v="West Sunset Boulevard" />
    <tag k="name" v="Blushington" />
    <tag k="opening_hours" v="Mo-Fr 08:00-19:00; Sa 08:00-20:00; Su 10:00-18:00" />
    <tag k="phone" v="+1-310-652-5874" />
    <tag k="shop" v="beauty" />
    <tag k="source" v="mapillary" />
    <tag k="website" v="https://www.blushington.com/store-locations/west-hollywood/" />
  </node>
```
we would like to change it to the following format:

```json
{"_id": "5133454776",
 "created": {"version": "1",
         "timestamp": "2017-09-27T17:01:30Z",
         "changeset": "52419516",
         "uid": "3232636",
         "user": "doktorpixel14"},
 "pos": [34.0928719, -118.3790788]
 "address": {"city": "West Hollywood",
         "housenumber": "8591",
         "postcode": 90069,
         "state": CA,
         "street": "West Sunset Boulevard"},
 "name": "Blushington",
 "opening_hours": "Mo-Fr 08:00-19:00; Sa 08:00-20:00; Su 10:00-18:00",
 "phone": "+1-310-652-5874",
 "shop": "beauty",
 "source": "mapillary"
 "website": "https://www.blushington.com/store-locations/west-hollywood/"}
 ```

### Part II: Clean the data
#### Audit the data

We now parse through the osm file with the python package ElementTree, in order to have an overview on the number of each type of element. Since the size of the file could be too large to put in memory, I use iterative parsing to obtain the branches of the file.

I define a function `find_tags()` which outputs the occurence of each tag name.

In [5]:
def find_tags(filename):
    tags = {}
    for event, element in ET.iterparse(filename, events=('start', )):
        ## if event == 'start', we parse from the top down
        ## if event == 'end', we parse from the bottom up
        ## For more information, please refer to the following post:
        ## http://blog.csdn.net/einstellung/article/details/74389145
        if element.tag not in tags:
            tags[element.tag] = 1
        else:
            tags[element.tag] += 1
    return tags

tags = find_tags("westhollywood.osm")

In [6]:
# I now define a sort function. 
# This function takes a dictionary as the input, 
# and outputs a sorted dataframe by ascending order of value.

def sort_dict(dict):
    df = pd.DataFrame(columns=['key', 'value'])
    dict_list = []
    
    ## the following loop turns a dictionary into a list of lists.
    for key, value in dict.iteritems():
        temp = [key,value]
        dict_list.append(temp)
        
    ## the following loop turns the list of lists into a dataframe.
    for i in range(0,len(dict)):
        df.loc[i] = dict_list[i]
        
    df['value'] = df['value'].astype('int64') ## change data type
    
    df = df.sort_values(by='value', ascending=False) ## sort according to the column 'value'
    return df

In [7]:
sort_dict(tags)

,key,value
2,nd,284742
0,node,256998
3,tag,163014
8,way,23891
1,member,8210
9,relation,687
4,bounds,1
5,note,1
6,meta,1
7,osm,1


#### Remark

As specified in [Openstreemap wiki](https://wiki.openstreetmap.org/wiki/Elements),

> Elements are the basic components of OpenStreetMap's conceptual data model of the physical world.
1. `<node>` defines points in space,
2. `<way>` defines linear features and area boundaries,
3. `<relation>` is sometimes used to explain how other elements work together.

`<nd>` and `<member>` are child tags.
- `<nd>` further defines the geometry of the `<way>`. [Example](https://wiki.openstreetmap.org/wiki/Way)
- `<member>` gives further information on the `<relation>`.

`<tag>` elements also give further information of the entity in question as a (key,value) pair. 

Now I want to summarize all the attributes, and to this end I define a function `find_attributes()`.

This function has similar structure to `find_tags()`, which outputs number of occurences of each attribute.

In [8]:
def find_attributes(filename):
    attributes = {}
    for event, element in ET.iterparse(filename, events=('start', 'end')):
        if event == 'start':
            for attr in element.attrib:
                if attr not in attributes:
                    attributes[attr] = 1
                else:
                    attributes[attr] += 1
    return attributes

attributes = find_attributes("westhollywood.osm")

In [9]:
#pprint.pprint(attributes)
sort_dict(attributes)

,key,value
16,ref,292952
10,version,281577
0,changeset,281576
17,id,281576
12,user,281576
18,uid,281576
5,timestamp,281576
8,lon,256998
14,lat,256998
6,k,163014


Now we focus on the attributes `k` and `v`, which are key components of the `<tag>` element

I define the following function `find_ks()`, which outputs the occurence of each `k` name.

In [10]:
def find_ks(filename):
    ks = {}
    for event, element in ET.iterparse(filename, events=('start', 'end')):
        if event == 'start':
            key = element.attrib.get('k')
            if key:
                if key not in ks:
                    ks[key] = 1
                else:
                    ks[key] += 1
    return ks

ks = find_ks("westhollywood.osm")

In [11]:
#pprint.pprint(ks)
sort_dict(ks)

,key,value
225,building,20710
24,ele,20261
136,height,20209
72,lacounty:bld_id,20205
235,lacounty:ain,20202
159,start_date,19473
172,building:units,17959
94,highway,2208
55,name,2175
26,source,978


The result suggests that the values of `k` attribute of the `<tag>` element contain the following symbols:

(1) uppercase letters.

(2) lowercase letters.

(3) "`_`"

(4) ":"

(5) numbers

(6) other problematic characters, such as empty space, period, etc.

There are also mixtures of those symbols. For simplicty, I summarize them into the following sets:
- lowercase and uppercase characters, possibly with "`_`". e.g. 'FIXME', "crossing", 'admin_level'
- lowercase and uppercase characters with merely one ":", and possibly with '`_`'. e.g. 'tiger:name_type_1',  
- others: e.g.  'name:historic:1999'
- problem characters (empty space, period etc.), e.g. "Business Category" 

I define a function `key_type()` which returns the occurence of each of the above tag categories.

In [12]:
# .complie() compiles a regular expression pattern into a regular expression object, 
#  which can be used for matching using its match() and search() methods.

lower_upper = re.compile(r'^([A-Za-z]|_)*$')
lower_upper_colon1 = re.compile(r'^([A-Za-z]|_)*:([A-Za-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        if lower_upper.search(element.attrib['k']): # .search() Returns None if no position in the string matches the pattern.
            keys['lower_upper'] += 1
        elif lower_upper_colon1.search(element.attrib['k']):
            keys['lower_upper_colon1'] += 1
        elif problemchars.search(element.attrib['k']):
            keys['problemchars'] += 1
        else:
            keys['other'] += 1
        
    return keys

def process_map_ks(filename):
    keys = {"lower_upper": 0, "lower_upper_colon1": 0, "problemchars": 0, "other": 0} # initialize the dictionary to store result.
    
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

keys = process_map_ks('westhollywood.osm')
pprint.pprint(keys)

{'lower_upper': 94452,
 'lower_upper_colon1': 67992,
 'other': 569,
 'problemchars': 1}


I could also identify the set of unique users, by the following codes.

In [13]:
def process_map_users(filename):
    users_unique = set()
    for _, element in ET.iterparse(filename):
        if (element.tag == 'node' or element.tag == 'way') and ("uid" in element.attrib):
            users_unique.add(element.attrib["uid"])

    return users_unique

users_unique = process_map_users("westhollywood.osm")
print 'Number of unique contributors:', len(users_unique)

Number of unique contributors: 251


#### Problems encountererd in the data

##### Problem 1. Nonstandard form of postal codes.
<!--I will now do the following:-->
<!--I will now do the following:-->
<!--I will now do the following:-->

In [14]:
def find_postcodes(filename):
    postcodes = {}
    for event, element in ET.iterparse(filename, events=('start', 'end')):
        if event == 'start':
            key = element.attrib.get('k') # .get('k') method for dictionary returns 0
                                          # when 'k' does not exist for the element in question
            if key == 'addr:postcode':
                pc_value = element.attrib.get('v')
                if pc_value not in postcodes:
                    postcodes[pc_value] = 1
                else:
                    postcodes[pc_value] += 1
    return postcodes

postcodes = find_postcodes('westhollywood.osm')
# sort_dict(postcodes)
pprint.pprint(postcodes)

{'90004': 1,
 '90027': 1,
 '90028': 4,
 '90036': 10,
 '90038': 5,
 '90046': 124,
 '90046-4101': 1,
 '90048': 9,
 '90069': 612,
 '90210': 47,
 'CA 90036': 1,
 'CA 90048': 1,
 'CA 90069': 2}


I proceed data cleaning in 2 steps.

Step 1. standardize the format to 5-digit string. 
  - e.g. 'CA 90036' -> '90036' and '90046-4101' -> '90046'

Step 2. add the city name behind the 5-digit postcode.
  - e.g. '90069' -> '90069, West Hollywood'
  
I start with Step 1:

In [15]:
# All the postal codes in California start with 9. 
def update_pcode_step1(string_in):
    string_out = string_in[string_in.find('9'): string_in.find('9')+5]
    return string_out

pcode_step1 = {}

for each in postcodes:
    pcode_step1[each] = update_pcode_step1(each)

pprint.pprint(pcode_step1)

{'90004': '90004',
 '90027': '90027',
 '90028': '90028',
 '90036': '90036',
 '90038': '90038',
 '90046': '90046',
 '90046-4101': '90046',
 '90048': '90048',
 '90069': '90069',
 '90210': '90210',
 'CA 90036': '90036',
 'CA 90048': '90048',
 'CA 90069': '90069'}


I now do Step 2.

Step 2 consists of two parts.

Part 1: I parse the website http://www.zipcodestogo.com/California/ to find the city name associated with the postal code.

Part 2: I append the city name behind the postcode.

In [16]:
# from bs4 import BeautifulSoup

URL = "http://www.zipcodestogo.com/California/"
r = requests.get(URL) # print(r.content) # bytes

soup = BeautifulSoup(r.content, 'lxml')  #print(soup.prettify())
table = soup.find_all('tr')

In [17]:
## here I find all the table rows which give information on the postal code
pcode_rows = []
for each in range(2,len(table)):
    if table[each].text.find('View Map') >=0:
        pcode_rows.append(each)

In [18]:
## here I extract the postal codes and the correpsonding cities.

A0 = {} # define a dictionary, 
        # where I store the postcode as the key, the city name as the value.

for each in pcode_rows:
    A0[str(table[each].text.split('\n')[1])] = \
           str(table[each].text.split('\n')[2])

In [19]:
def update_pcode_step2(string_in):
    if string_in in A0.keys():
        string_out = string_in + ', ' + A0[string_in]
    return string_out

pcode_step = {}

for each in postcodes:
    better_name = update_pcode_step1(each)
    pcode_step[each] = update_pcode_step2(better_name)

pprint.pprint(pcode_step)

{'90004': '90004, Los Angeles',
 '90027': '90027, Los Angeles',
 '90028': '90028, Los Angeles',
 '90036': '90036, Los Angeles',
 '90038': '90038, Los Angeles',
 '90046': '90046, Los Angeles',
 '90046-4101': '90046, Los Angeles',
 '90048': '90048, Los Angeles',
 '90069': '90069, West Hollywood',
 '90210': '90210, Beverly Hills',
 'CA 90036': '90036, Los Angeles',
 'CA 90048': '90048, Los Angeles',
 'CA 90069': '90069, West Hollywood'}


##### Problem 2: Nonstandard form of phone number

In [20]:
def find_phones(filename):
    phones = {}
    for event, element in ET.iterparse(filename, events=('start', 'end')):
        if event == 'start':
            key = element.attrib.get('k') # important, .get('k') method for dictionary returns 0
                                          # when 'k' does not exist for the element in question
            if key == 'phone':
                phone_value = element.attrib.get('v')
                if phone_value not in phones:
                    phones[phone_value] = 1
                else:
                    phones[phone_value] += 1
    return phones

phones = find_phones('westhollywood.osm')
pprint.pprint(phones)

{'(323) 464-2989': 1,
 '(323) 654-4411': 1,
 '(323) 654-6686': 1,
 '(323) 874-6700': 1,
 '(323) 934-1121': 1,
 '+1 (310) 980 8064': 1,
 '+1 (323) 857-1882': 1,
 '+1 (323) 928-3002': 1,
 '+1 (323) 944-0855': 1,
 '+1 310 3600916': 1,
 '+1 310-652-5091': 1,
 '+1 310-854-3488': 1,
 '+1 323 654 7125': 1,
 '+1 323 6548271': 1,
 '+1 323 747 1388': 1,
 '+1 323-654-4222': 1,
 '+1 323-654-8713': 1,
 '+1- 323-978-2170': 1,
 '+1-1-310-424-1600': 1,
 '+1-213-224-4153': 1,
 '+1-213-261-3622': 1,
 '+1-213-291-0466': 1,
 '+1-310-228-1051': 1,
 '+1-310-246-0030': 1,
 '+1-310-246-2560': 1,
 '+1-310-247-1112': 1,
 '+1-310-248-3750': 1,
 '+1-310-271-2345': 1,
 '+1-310-271-5670': 1,
 '+1-310-271-6261': 1,
 '+1-310-271-9664': 1,
 '+1-310-273-3265': 1,
 '+1-310-273-3303': 1,
 '+1-310-273-5126': 1,
 '+1-310-273-9399': 1,
 '+1-310-273-9993': 1,
 '+1-310-274-1395': 1,
 '+1-310-274-1715': 1,
 '+1-310-274-2326': 1,
 '+1-310-274-5313': 1,
 '+1-310-274-6201': 1,
 '+1-310-275-0055': 1,
 '+1-310-275-2661': 1,
 '+1-31

The standard format that I use is `+1-213-224-4153`, that is 

1. the number starts with '+1';
2. the number is of 11 digits;
3. the '-' appears at the 2nd, 5th, 8th digit place.

By the following codes, I identify all the bad formats.

In [21]:
phones_bad = {} ## problematique phone numbers.
for each in phones:
    if (each.startswith('+1-') != True) or \
                (len(each) != 15):
            if (each not in phones_bad):
                phones_bad[each] = 1
            else:
                phones_bad[each] += 1
phones_bad

{'(323) 464-2989': 1,
 '(323) 654-4411': 1,
 '(323) 654-6686': 1,
 '(323) 874-6700': 1,
 '(323) 934-1121': 1,
 '+1 (310) 980 8064': 1,
 '+1 (323) 857-1882': 1,
 '+1 (323) 928-3002': 1,
 '+1 (323) 944-0855': 1,
 '+1 310 3600916': 1,
 '+1 310-652-5091': 1,
 '+1 310-854-3488': 1,
 '+1 323 654 7125': 1,
 '+1 323 6548271': 1,
 '+1 323 747 1388': 1,
 '+1 323-654-4222': 1,
 '+1 323-654-8713': 1,
 '+1- 323-978-2170': 1,
 '+1-1-310-424-1600': 1,
 '+1-323-650-057': 1,
 '+1-800-PINKDOT': 1,
 '+13239931700': 1,
 '0013239366154': 1,
 '013239514800': 1,
 '1-323-871-8318': 1,
 '1-323-874-7924': 1,
 '310 289 2000': 1,
 '310-720-3809': 1,
 '323-379-2091': 1,
 '323-415-6860': 1,
 '323-424-7731': 1,
 '323-540-4551': 1,
 '323-697-5338': 1,
 '323-931-1466': 1,
 '323-937-2801': 1,
 '323-988-1119': 1,
 '855-888-5575': 1}

These bad formats may belong to one or several of the following categories:

1. lack of '+1' at the beginning. e.g.'323-697-5338'
2. space instead of hyphen. e.g.  '+1 323 654 7125'
3. redudant spaces. e.g. '+1 323-654-8713', '+1- 323-978-2170'.
4. parenthese. e.g.  '+1 (323) 944-0855', '(323) 654-4411'
5. lack of hyphens. e.g. '+13239931700'
6. redudant zeros. e.g. '0013239366154','013239514800' 
7. one digit missing. e.g. '+1-323-650-057'
8. service hotline. e.g. '+1-800-PINKDOT' (<- This is a acceptable)

I will concentrate on (1)-(6). Notice that (8) is of correct format, and (7) could not be dealt with without further information.


#### I adopt the following strategies:

Step 1. remove all the characters. e.g. hyphens, spaces, parentesis

Step 2. change the pure number to the standard format.

By the following code, I execute Step 1.

In [22]:
phones_step1 = {}

def update_phones_step1(string_in):
    problemchars = ' ()+-'
    for ch in problemchars:
        string_in = string_in.replace(ch,'')
    return string_in

for key in phones_bad:
    key_original = key
    phones_step1[key_original] = update_phones_step1(key)

phones_step1

{'(323) 464-2989': '3234642989',
 '(323) 654-4411': '3236544411',
 '(323) 654-6686': '3236546686',
 '(323) 874-6700': '3238746700',
 '(323) 934-1121': '3239341121',
 '+1 (310) 980 8064': '13109808064',
 '+1 (323) 857-1882': '13238571882',
 '+1 (323) 928-3002': '13239283002',
 '+1 (323) 944-0855': '13239440855',
 '+1 310 3600916': '13103600916',
 '+1 310-652-5091': '13106525091',
 '+1 310-854-3488': '13108543488',
 '+1 323 654 7125': '13236547125',
 '+1 323 6548271': '13236548271',
 '+1 323 747 1388': '13237471388',
 '+1 323-654-4222': '13236544222',
 '+1 323-654-8713': '13236548713',
 '+1- 323-978-2170': '13239782170',
 '+1-1-310-424-1600': '113104241600',
 '+1-323-650-057': '1323650057',
 '+1-800-PINKDOT': '1800PINKDOT',
 '+13239931700': '13239931700',
 '0013239366154': '0013239366154',
 '013239514800': '013239514800',
 '1-323-871-8318': '13238718318',
 '1-323-874-7924': '13238747924',
 '310 289 2000': '3102892000',
 '310-720-3809': '3107203809',
 '323-379-2091': '3233792091',
 '323-4

For step 2,

1. If the entry starts with 0 or 1, I will remove any 0 or 1 at the beginning.I save the result for the next step.

2. Then, I take the above result, put '+1-' at the beginning, and insert '-' respectively at 5th and 8th place. 

In [23]:
def update_phones_step2(string_in):
    while (string_in[0] == '0') or (string_in[0] == '1'):
        string_in = string_in[1:len(string_in)]

    if re.compile(r'[A-Z]+$').search(string_in):
        string_out = '+1-' + string_in[0:3] + '-' +\
        string_in[3:len(string_in)]
    else:
        string_out = '+1-' + string_in[0:3] + '-' +\
        string_in[3:6] + '-' + string_in[6:len(string_in)]
    return string_out

phones_step2 = {}

for each in phones_step1:
    phones_step2[each] = update_phones_step2(phones_step1[each])

phones_step2


{'(323) 464-2989': '+1-323-464-2989',
 '(323) 654-4411': '+1-323-654-4411',
 '(323) 654-6686': '+1-323-654-6686',
 '(323) 874-6700': '+1-323-874-6700',
 '(323) 934-1121': '+1-323-934-1121',
 '+1 (310) 980 8064': '+1-310-980-8064',
 '+1 (323) 857-1882': '+1-323-857-1882',
 '+1 (323) 928-3002': '+1-323-928-3002',
 '+1 (323) 944-0855': '+1-323-944-0855',
 '+1 310 3600916': '+1-310-360-0916',
 '+1 310-652-5091': '+1-310-652-5091',
 '+1 310-854-3488': '+1-310-854-3488',
 '+1 323 654 7125': '+1-323-654-7125',
 '+1 323 6548271': '+1-323-654-8271',
 '+1 323 747 1388': '+1-323-747-1388',
 '+1 323-654-4222': '+1-323-654-4222',
 '+1 323-654-8713': '+1-323-654-8713',
 '+1- 323-978-2170': '+1-323-978-2170',
 '+1-1-310-424-1600': '+1-310-424-1600',
 '+1-323-650-057': '+1-323-650-057',
 '+1-800-PINKDOT': '+1-800-PINKDOT',
 '+13239931700': '+1-323-993-1700',
 '0013239366154': '+1-323-936-6154',
 '013239514800': '+1-323-951-4800',
 '1-323-871-8318': '+1-323-871-8318',
 '1-323-874-7924': '+1-323-874-792

#### Problem 3: Abreviated Street names

We could observe that most of the street names are either abbreviated or mistyped, e.g.,
- Santa Monica Blvd. => Santa Monica Boulevard.

I now follow the procedure in the course to update the street names.

In [24]:
# a regular expression which matches a string with a word ending optionally with a period. 
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Avenue", "Boulevard", "Circle", "Commons", "Court", \
            "Drive", "Lane", "Parkway","Place", "Plaza", \
            "Road", "Square", "Street", "Trail", "Way", "West"]

In [25]:
filename = 'westhollywood.osm'

def audit_string(string_in, re_pattern, expected_types, dict_output):
    '''
    Whenever we detect unexpected a string entry, 
    we store it in a dictionary.
    '''
    m = re_pattern.search(string_in)
    if m:
        matched = m.group()
        if matched not in expected_types:
            dict_output[matched].add(string_in)
            
def audit(file_in, filter, re_pattern, expected_types):
    
    file = open(file_in, "r")
    result = defaultdict(set)
    
    for event, element in ET.iterparse(file, events=("start",)):
        if element.tag == "node" or element.tag == "way":
            for tag in element.iter("tag"): 
                ## iterate over child tags
                if filter(tag):
                    audit_string(tag.attrib['v'], re_pattern, expected_types, result)
    file.close()
    return result

def is_street_name(elem):
    '''
    check whether the tag contains information on street type.
    '''
    return (elem.attrib['k'] == "addr:street")

street_types_bad = \
audit(filename, filter = is_street_name, re_pattern = street_type_re, \
      expected_types = expected)

pprint.pprint(dict(street_types_bad))

{'Ave': set(['N La Brea Ave']),
 'Blvd': set(['7290 Beverly Blvd',
              'North Robertson Blvd',
              'Santa Monica Blvd',
              'Sunset Blvd']),
 'Blvd.': set(['North Robertson Blvd.', 'Santa Monica Blvd.']),
 'Mansfield': set(['North Mansfield']),
 'avenue': set(['north la brea avenue']),
 'blvd': set(['sunset blvd'])}


Now I will map the listed bad street types into the correct ones. The mapping is as follows.

In [26]:
street_types_mapping = \
    {
        "avenue" : "Avenue",
        "Ave" : "Avenue",
        "blvd" : "Boulevard",
        #"Boulvard" : "Boulevard",
        "Blvd" : "Boulevard",
        "Blvd." : "Boulevard"#,
        #"Dr" : "Drive",
        #"Dr." : "Drive",
        #"Rd" : "Road",
        #"St" : "Street"
    }

In [27]:
def update_street(string_in, mapping, re_pattern):
    m = re_pattern.search(string_in)
    if m:
        matched = m.group()
        if matched in mapping:
            string_in = re.sub(re_pattern,mapping[matched],string_in)
    return string_in

street_types_bad_join = '|'.join(street_types_bad.keys())
    
street_type_step1_re = \
re.compile(r'\b(' + street_types_bad_join.replace('.','') + r')\b\.?', re.IGNORECASE)

for key,values in street_types_bad.iteritems():
    for name in values:
        name_step1 = update_street(name, street_types_mapping, \
                                   street_type_step1_re)
        print name, "=>", name_step1

sunset blvd => sunset Boulevard
North Mansfield => North Mansfield
Sunset Blvd => Sunset Boulevard
Santa Monica Blvd => Santa Monica Boulevard
7290 Beverly Blvd => 7290 Beverly Boulevard
North Robertson Blvd => North Robertson Boulevard
N La Brea Ave => N La Brea Avenue
north la brea avenue => north la brea Avenue
North Robertson Blvd. => North Robertson Boulevard
Santa Monica Blvd. => Santa Monica Boulevard


We notice that the directions north, south, east and west are also sometimes abbreviated. We now deal with it.

In [28]:
direction_re = re.compile(r'([NSEW]|north|south|east|west)\b\.?') ## goo.gl/mtFRPP

expected_directions = ['North', 'South', 'East', 'West']

directions_bad = audit(filename, is_street_name, direction_re, expected_directions)

pprint.pprint(dict(directions_bad))

{'N': set(['N La Brea Ave']), 'north': set(['north la brea avenue'])}


In [29]:
directions_mapping = \
    {
        #"E" : "East",
        #"E." : "East",
        "N" : "North",
        "north": "North"
        #"N." : "North",
        #"S" : "South",
        #"S." : "South",
        #"W" : "West",
        #"W." : "West"
    }

## There are two steps to follow:
## First step: I update the street type.
## Second step: I update the direction, based on the result of first step.

directions_bad_join = "|".join(directions_mapping.keys())
directions_step1_re = \
re.compile(r'\b(' + directions_bad_join + r')\b\.?', re.IGNORECASE)

for key, values in directions_bad.iteritems():
    for name in values:
        name_step1 = \
        update_street(name, street_types_mapping, street_type_step1_re)
        name_step2 = \
        update_street(name_step1, directions_mapping, directions_step1_re)
        print name, "=>", name_step1, "=>", name_step2

north la brea avenue => north la brea Avenue => North la brea Avenue
N La Brea Ave => N La Brea Avenue => North La Brea Avenue


#### Finally, I turn XML into CSV format, which is later imported into Sqlite.

In [30]:
OSM_PATH = "westhollywood.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

# LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

# SCHEMA = schema.Schema
# SCHEMA = schema.Schema(my_schema)

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

In [31]:
def update_tag_children(tag,ele,result,attrib_filter):
    # Iterate tag children
    for tag in ele.iter("tag"):
        if not attrib_filter.search(tag.attrib['k']):
            tags_each = {}

            # Single colon beginning
            if tag.attrib['k'].find(':') > 0:
                sub_attr = tag.attrib['k'].split(':', 1)
                tags_each['key'] = sub_attr[1]
                tags_each['type'] = sub_attr[0]
                
                if tag.attrib.get('k') == 'addr:postcode':
                    pcode_step1 = update_pcode_step1(tag.attrib['v'])
                    tags_each['value'] = update_pcode_step2(pcode_step1)
                elif tag.attrib.get('k') == 'addr:street':
                    name_step1 = \
                    update_street(tag.attrib['v'], \
                                  street_types_mapping, street_type_step1_re)
                    name_step2 = \
                    update_street(name_step1, \
                                  directions_mapping, directions_step1_re)

                    tags_each['value'] = name_step2
                else:
                    tags_each['value'] = tag.attrib['v']
            # Tags with no colon
            elif tag.attrib['k'].find(':') == -1:
                tags_each['key'] = tag.attrib['k']
                tags_each['type'] = 'regular'
                    
                if tag.attrib.get('k') == 'phone':
                    phone_step1 = update_phones_step1(tag.attrib['v'])
                    tags_each['value'] = update_phones_step2(phone_step1)
                else:
                    tags_each['value'] = tag.attrib['v']
                
            tags_each['id'] = ele.attrib['id']
                    
            result.append(tags_each)
    return result

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements
    
    # MY CODE HERE    
    if element.tag == "node":
        for a in element.attrib:
            if a in node_attr_fields:
                node_attribs[a] = element.attrib[a]
        
        # Iterate tag children
        update_tag_children(tag = 'tag',ele = element, \
                            result = tags, attrib_filter = problem_chars)
    
    if element.tag == "way":
        for a in element.attrib:
            if a in way_attr_fields:
                way_attribs[a] = element.attrib[a]
        
        # Iterate tag children
        update_tag_children(tag = 'tag',ele = element, \
                            result = tags, attrib_filter = problem_chars)

        count_nd = 0
        for nd in element.iter("nd"):
            nodes_each = {}
            nodes_each['id'] = element.attrib['id']
            nodes_each['node_id'] = nd.attrib['ref']
            nodes_each['position']= count_nd
            
            way_nodes.append(nodes_each)
            count_nd += 1
            
    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

In [32]:
# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


In [33]:
# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in): #, validate
    """Iteratively process each XML element and write to csv(s)"""
    
    ## Notice that we must use 'wb', instead of 'w'
    ## , See link: https://bit.ly/2GObbtj
    
    ## validation problem: https://github.com/marshmallow-code/marshmallow/issues/120
    
    with codecs.open(NODES_PATH, 'wb') as nodes_file, \
    codecs.open(NODE_TAGS_PATH, 'wb') as nodes_tags_file, \
    codecs.open(WAYS_PATH, 'wb') as ways_file, \
    codecs.open(WAY_NODES_PATH, 'wb') as way_nodes_file, \
    codecs.open(WAY_TAGS_PATH, 'wb') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        #validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                #if validate is True:
                #    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH) #, validate=True)

By the above code, we generate the csv files.

Below is an overview of the files.

> Size of the file ....... 58,913 KB

> nodes.csv .............. 23,462 KB

> nodes_tags.csv ......... 293 KB

> ways.csv ............... 1,595 KB

> ways_tags.csv .......... 6,761 KB

> ways_nodes.cv .......... 4,979 KB

Now I will use python to load the csv files in the Sqlite database. Please refer to another file named `Investigate data using sql.ipynb`.